<a href="https://colab.research.google.com/github/jangInw007/JENZY_Project/blob/main/JENZY_is_perfect_project_Basicpro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# path_to_stopandsearch = '/content/drive/MyDrive/JENZY/Data/2025-01'

In [7]:
import pandas as pd

In [17]:
# import os

# directory_path = '/content/drive/MyDrive/JENZY/Data/2025-01'
# files_in_directory = os.listdir(directory_path)

# print(f"Files in directory: {directory_path}")
# for file in files_in_directory:
#     print(file)

Files in directory: /content/drive/MyDrive/JENZY/Data/2025-01
2025-01-hampshire-stop-and-search.csv
2025-01-suffolk-street.csv
2025-01-avon-and-somerset-stop-and-search.csv
2025-01-gloucestershire-outcomes.csv
2025-01-cheshire-stop-and-search.csv
2025-01-devon-and-cornwall-stop-and-search.csv
2025-01-west-yorkshire-street.csv
2025-01-northamptonshire-outcomes.csv
2025-01-gwent-street.csv
2025-01-staffordshire-street.csv
2025-01-dyfed-powys-street.csv
2025-01-warwickshire-stop-and-search.csv
2025-01-north-wales-stop-and-search.csv
2025-01-northumbria-outcomes.csv
2025-01-cheshire-outcomes.csv
2025-01-norfolk-street.csv
2025-01-lincolnshire-street.csv
2025-01-derbyshire-stop-and-search.csv
2025-01-south-wales-outcomes.csv
2025-01-cumbria-street.csv
2025-01-merseyside-street.csv
2025-01-cleveland-outcomes.csv
2025-01-cambridgeshire-stop-and-search.csv
2025-01-dyfed-powys-stop-and-search.csv
2025-01-cumbria-outcomes.csv
2025-01-thames-valley-street.csv
2025-01-leicestershire-street.csv
202

In [23]:
# # Use len() to get the number of items in the list
# num_files = len(files_in_directory)
# print(f"Number of files in the directory: {num_files}")

Number of files in the directory: 123


In [24]:
import os

base_path = '/content/drive/MyDrive/JENZY/Data/'
monthly_files = {}

for month in range(1, 6):  # Iterate from month 1 to 5
    month_str = f'2025-{month:02d}'  # Format month as 01, 02, etc.
    month_path = os.path.join(base_path, month_str)

    if os.path.exists(month_path) and os.path.isdir(month_path):
        files_in_month = os.listdir(month_path)
        monthly_files[month_str] = files_in_month
        print(f"Files in {month_str}:")
        for file in files_in_month:
            print(file)
        print("-" * 20) # Separator for clarity
    else:
        print(f"Directory not found: {month_path}")

# You can now access the list of files for each month using monthly_files['2025-01'], monthly_files['2025-02'], etc.

Files in 2025-01:
2025-01-hampshire-stop-and-search.csv
2025-01-suffolk-street.csv
2025-01-avon-and-somerset-stop-and-search.csv
2025-01-gloucestershire-outcomes.csv
2025-01-cheshire-stop-and-search.csv
2025-01-devon-and-cornwall-stop-and-search.csv
2025-01-west-yorkshire-street.csv
2025-01-northamptonshire-outcomes.csv
2025-01-gwent-street.csv
2025-01-staffordshire-street.csv
2025-01-dyfed-powys-street.csv
2025-01-warwickshire-stop-and-search.csv
2025-01-north-wales-stop-and-search.csv
2025-01-northumbria-outcomes.csv
2025-01-cheshire-outcomes.csv
2025-01-norfolk-street.csv
2025-01-lincolnshire-street.csv
2025-01-derbyshire-stop-and-search.csv
2025-01-south-wales-outcomes.csv
2025-01-cumbria-street.csv
2025-01-merseyside-street.csv
2025-01-cleveland-outcomes.csv
2025-01-cambridgeshire-stop-and-search.csv
2025-01-dyfed-powys-stop-and-search.csv
2025-01-cumbria-outcomes.csv
2025-01-thames-valley-street.csv
2025-01-leicestershire-street.csv
2025-01-west-mercia-outcomes.csv
2025-01-lancas

In [31]:
file_type_counts = {}

for month, files in monthly_files.items():
    stop_and_search_count = 0
    street_count = 0
    outcomes_count = 0
    other_count = 0
    total_count = len(files) # Calculate total files in the month

    for file in files:
        if 'stop-and-search' in file:
            stop_and_search_count += 1
        elif 'street' in file:
            street_count += 1
        elif 'outcomes' in file:
            outcomes_count += 1
        else:
            other_count += 1

    file_type_counts[month] = {
        'stop-and-search': stop_and_search_count,
        'street': street_count,
        'outcomes': outcomes_count,
        'other': other_count,
        'total': total_count # Add total count to the dictionary
    }

print("File counts per month and type:")
for month, counts in file_type_counts.items():
    print(f"\n{month}:")
    print(f"  total: {counts['total']}") # Print total count first
    for file_type, count in counts.items():
        if file_type != 'total': # Avoid printing total twice
            print(f"  {file_type}: {count}")

File counts per month and type:

2025-01:
  total: 123
  stop-and-search: 38
  street: 43
  outcomes: 42
  other: 0

2025-02:
  total: 96
  stop-and-search: 38
  street: 28
  outcomes: 30
  other: 0

2025-03:
  total: 53
  stop-and-search: 37
  street: 5
  outcomes: 11
  other: 0

2025-04:
  total: 52
  stop-and-search: 37
  street: 5
  outcomes: 10
  other: 0

2025-05:
  total: 49
  stop-and-search: 36
  street: 5
  outcomes: 8
  other: 0


In [35]:
# Count the number of files in each category
num_stop_and_search = len(all_stop_and_search_files)
num_street = len(all_street_files)
num_outcomes = len(all_outcomes_files)
num_other = len(all_other_files)

print(f"Number of Stop and Search Files: {num_stop_and_search}")
print(f"Number of Street Files: {num_street}")
print(f"Number of Outcomes Files: {num_outcomes}")
if num_other > 0:
    print(f"Number of Other Files: {num_other}")

Number of Stop and Search Files: 186
Number of Street Files: 86
Number of Outcomes Files: 101


In [46]:

#Real HW4
# นำเข้าโมดูล drive จาก google.colab เพื่อเข้าถึง Google Drive
from google.colab import drive
# นำเข้าไลบรารี pandas สำหรับจัดการและวิเคราะห์ข้อมูล
import pandas as pd
# นำเข้าโมดูล os สำหรับโต้ตอบกับระบบปฏิบัติการ เช่น การแสดงรายการไฟล์
import os
# นำเข้า defaultdict จาก collections เพื่อช่วยในการสร้าง dictionary ที่มีค่าเริ่มต้น
from collections import defaultdict

# เชื่อมต่อ Google Drive เข้ากับสภาพแวดล้อม Colab
drive.mount('/content/drive')

# กำหนดพาธหลักไปยังโฟลเดอร์ข้อมูลใน Google Drive
# base_path = '/content/drive/MyDrive/JENZY/Data/2025-01'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
base_path = '/content/drive/MyDrive/JENZY/Data'

In [56]:

# สร้างลิสต์ว่างๆ เพื่อเก็บ DataFrame จากไฟล์ CSV ทั้งหมด
all_data = []

# วนลูปผ่านแต่ละโฟลเดอร์ (แทนแต่ละเดือน) ในพาธข้อมูลหลัก อ่านชื่อไฟล์ทุกตัวที่อยู่ในไฟล์ base_path
for month in os.listdir(base_path):
    # สร้างพาธแบบเต็มไปยังโฟลเดอร์ของเดือนปัจจุบัน
    #/content/drive/MyDrive/Python&DataViz2025/Data/2025-02
    month_path = os.path.join(base_path, month)
#-------------------------------------------------------------------------------
    # ตรวจสอบว่ารายการปัจจุบันเป็นไดเรกทอรี (โฟลเดอร์) หรือไม่
    if not os.path.isdir(month_path):
        # หากไม่ใช่ไดเรกทอรี ให้ข้ามไปยังรายการถัดไป
        continue
    # วนลูปผ่านแต่ละไฟล์ในโฟลเดอร์ของเดือนปัจจุบัน
    for file in os.listdir(month_path):
        # ตรวจสอบว่า 'stop-and-search' อยู่ในชื่อไฟล์หรือไม่
        if 'stop-and-search' not in file:
            # หากไฟล์ไม่ใช่ไฟล์ stop-and-search ให้ข้ามไปยังไฟล์ถัดไป
            continue
        # สร้างพาธแบบเต็มไปยังไฟล์ปัจจุบัน
        filepath = os.path.join(month_path, file)
#-------------------------------------------------------------------------------
        # ใช้บล็อก try-except เพื่อจัดการข้อผิดพลาดที่อาจเกิดขึ้นระหว่างการอ่านไฟล์
        try:
            # อ่านไฟล์ CSV เข้ามาเก็บไว้ใน pandas DataFrame
            df = pd.read_csv(filepath)
            # แยกชื่อเมืองออกจากชื่อไฟล์
            # โดยการแยกชื่อไฟล์ด้วย '-' และนำส่วนที่อยู่หลังปีและเดือน
            # จากนั้นรวมส่วนที่เหลือด้วย '-' และลบช่องว่างที่นำหน้า/ตามหลัง
            file_parts = file.replace('-stop-and-search.csv', '').split('-')
            #['2025', '01', 'bedfordshire']
            #['2025', '01', 'west', 'midlands']
            city = '-'.join(file_parts[2:]).strip()
            # เพิ่มคอลัมน์ใหม่ 'City' ไปยัง DataFrame และเติมชื่อเมืองที่แยกได้ลงไป
            df['City'] = city
            # เพิ่ม DataFrame ที่ประมวลผลแล้วเข้าในลิสต์ all_data
            all_data.append(df)
        # ดักจับข้อยกเว้นใดๆ ที่เกิดขึ้นระหว่างการประมวลผลไฟล์
        except Exception as e:
            # พิมพ์ข้อความแสดงข้อผิดพลาดหากไม่สามารถอ่านไฟล์ได้
            print(f"อ่านไฟล์ไม่ได้: {filepath} : {e}")


In [57]:
# Combine all DataFrames in the list into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Get the unique city names from the 'City' column
unique_cities_in_data = combined_df['City'].unique()

print("Unique cities found in the combined data:")
for city in unique_cities_in_data:
    print(city)

print(f"\nTotal number of unique cities in the combined data: {len(unique_cities_in_data)}")

Unique cities found in the combined data:
hampshire
avon-and-somerset
cheshire
devon-and-cornwall
warwickshire
north-wales
derbyshire
cambridgeshire
dyfed-powys
suffolk
west-midlands
norfolk
durham
west-mercia
cumbria
city-of-london
west-yorkshire
leicestershire
northumbria
sussex
kent
lancashire
nottinghamshire
btp
thames-valley
merseyside
surrey
essex
metropolitan
northamptonshire
dorset
bedfordshire
cleveland
south-wales
gloucestershire
south-yorkshire
hertfordshire
staffordshire

Total number of unique cities in the combined data: 38


----------------------------------------------------------------

#รวม stop_and_search 2025-01

In [82]:
import os
import pandas as pd

month_path = '/content/drive/MyDrive/JENZY/Data/2025-01'
stop_and_search_dfs = []

if os.path.isdir(month_path):
    for file in os.listdir(month_path):
        if 'stop-and-search.csv' in file:
            filepath = os.path.join(month_path, file)
            try:
                df = pd.read_csv(filepath)
                stop_and_search_dfs.append(df)
            except Exception as e:
                print(f"Error reading file: {filepath} : {e}")
else:
    print(f"Directory not found: {month_path}")

if stop_and_search_dfs:
    combined_stop_and_search_202501 = pd.concat(stop_and_search_dfs, ignore_index=True)
    print("Combined DataFrame from 2025-01 stop-and-search files:")
    display(combined_stop_and_search_202501.tail(5))
else:
    print("No stop-and-search files found in 2025-01 directory.")

Combined DataFrame from 2025-01 stop-and-search files:


,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
37943,Person search,2025-01-31T23:07:00+00:00,NaN,NaN,52.633024,-1.693795,Male,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False
37944,Vehicle search,2025-01-31T23:34:00+00:00,NaN,NaN,53.041074,-2.234089,Other,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,True,NaN
37945,Person search,2025-01-31T23:35:00+00:00,NaN,NaN,53.041074,-2.234089,Female,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False
37946,Person search,2025-01-31T23:35:00+00:00,NaN,NaN,53.041074,-2.234089,Male,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False
37947,Vehicle search,2025-01-31T23:35:00+00:00,NaN,NaN,53.041074,-2.234089,Other,NaN,NaN,NaN,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,NaN


# รวม Outcomes 2025-01

In [83]:
import os
import pandas as pd

month_path = '/content/drive/MyDrive/JENZY/Data/2025-01'
outcomes_dfs = []

if os.path.isdir(month_path):
    for file in os.listdir(month_path):
        if 'outcomes.csv' in file:
            filepath = os.path.join(month_path, file)
            try:
                df = pd.read_csv(filepath)
                outcomes_dfs.append(df)
            except Exception as e:
                print(f"Error reading file: {filepath} : {e}")
else:
    print(f"Directory not found: {month_path}")

if outcomes_dfs:
    combined_outcomes_202501 = pd.concat(outcomes_dfs, ignore_index=True)
    print("Combined DataFrame from 2025-01 outcomes files:")
    display(combined_outcomes_202501.tail(5))
else:
    print("No outcomes files found in 2025-01 directory.")

Combined DataFrame from 2025-01 outcomes files:


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Outcome type
419686,75d6a11445118bb5f13b55580c0775c5e43a0d704d0aa0...,2025-01,West Yorkshire Police,West Yorkshire Police,-1.531599,53.801162,On or near Mabgate,E01033031,Leeds 064E,Suspect charged
419687,0f3b8eb7d26d37346df702d544fa4e6c00f3f78158493c...,2025-01,West Yorkshire Police,West Yorkshire Police,-1.664544,53.798892,On or near Back Richardshaw Lane,E01011587,Leeds 062B,Offender given a caution
419688,389d456f4a020acae4c0b3d0620218a53ae108060e4e56...,2025-01,West Yorkshire Police,West Yorkshire Police,-1.574524,53.775355,On or near Police Station,E01011369,Leeds 090E,Further action is not in the public interest
419689,008fe18a501c0109172f79e4c43d69c947dc2226063750...,2025-01,West Yorkshire Police,West Yorkshire Police,-1.552667,53.726057,On or near Westerton Walk,E01011538,Leeds 107B,Further action is not in the public interest
419690,957a57915cac58e1bab5fc39119a6305ce5418596dff05...,2025-01,West Yorkshire Police,West Yorkshire Police,-1.299449,53.593057,On or near Wesley Street,E01011869,Wakefield 043B,Suspect charged


# รวม Street 2025-01

In [84]:
import os
import pandas as pd

month_path = '/content/drive/MyDrive/JENZY/Data/2025-01'
street_dfs = []

if os.path.isdir(month_path):
    for file in os.listdir(month_path):
        if 'street.csv' in file:
            filepath = os.path.join(month_path, file)
            try:
                df = pd.read_csv(filepath)
                street_dfs.append(df)
            except Exception as e:
                print(f"Error reading file: {filepath} : {e}")
else:
    print(f"Directory not found: {month_path}")

if street_dfs:
    combined_street_202501 = pd.concat(street_dfs, ignore_index=True)
    print("Combined DataFrame from 2025-01 outcomes files:")
    display(combined_street_202501.tail(5))
else:
    print("No street files found in 2025-01 directory.")

Combined DataFrame from 2025-01 outcomes files:


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
443695,3e413cfaba2318b700f25c709f473ccef5d7c9fa249255...,2025-01,Cheshire Constabulary,Cheshire Constabulary,-2.570489,53.342101,On or near Camberley Heath Drive,E01012503,Warrington 024D,Violence and sexual offences,Under investigation,NaN
443696,106c44c2c2327f5c6350723ee3745428d72c0cea2d00ad...,2025-01,Cheshire Constabulary,Cheshire Constabulary,-2.568192,53.330858,On or near Northwich Road,E01012503,Warrington 024D,Violence and sexual offences,Further investigation is not in the public int...,NaN
443697,76fb6bf83c3d5d5812a40c8942488e043d8d14d9d42fad...,2025-01,Cheshire Constabulary,Cheshire Constabulary,-2.568185,53.342786,On or near Henbury Gardens,E01012503,Warrington 024D,Other crime,Investigation complete; no suspect identified,NaN
443698,f16e8aa75218ed74181268ac4761c3b4f8bc222600c8d1...,2025-01,Cheshire Constabulary,Cheshire Constabulary,-2.578321,53.342620,On or near Shepcroft Lane,E01012452,Warrington 025D,Vehicle crime,Investigation complete; no suspect identified,NaN
443699,1ef86a07a9df3dac37575b2553ec0e0358ff2039c5b09e...,2025-01,Cheshire Constabulary,Cheshire Constabulary,-2.598305,53.469595,On or near Summercroft Close,E01006346,Wigan 038E,Violence and sexual offences,Under investigation,NaN


1.อธิบายข้อมูล ว่าข้อมูลที่ใช้คืออะไร

1.ดูว่ามีไฟล์กี่ประเภท เช่น stop and search...

2.มีกี่เมืองในแต่ละเดือน

3.แต่ละประเภท ในแต่ละ เดือน มีเมืองเท่ากันไหม

2.เตรียมข้อมูล ทำความสะอาดข้อมูล

3.ข้อมูลจริงที่จะนำไปทำ dashboard

- ทำกราฟก่อนไปlooker

Visualize_Table_Data.ipynb
- Headmap
- pandas: parallel coordinate
- pandas: scatter matrix
- Scatter plot

Visualize_Data_Distribution.ipynb
- ใช้ขนาดของ marker แสดงค่าของ feature
- ตัวแปร Marker ใช้ในการเปลี่ยนหน้าตาของ marker
- การ plot กราฟหลายๆอันซ้อนกัน
- Plot-Legend

Visualize_Data_Comparison_2025.ipynb
- Bar chart (กราฟแท่ง)
- Stacked barchart
- Group barchart
- tree map
- Radar Chart